<a href="https://colab.research.google.com/github/andres-merino/Curso-FundMachineLearning/blob/main/3-Notebooks/03-3-Validacion-Cruzada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i0.wp.com/see-ec.org/wordpress/wp-content/uploads/2023/01/LOGO-SEE-texto-blanco-3.png?w=1356&ssl=1" width="200">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#008dc3" size="7">|</font><br>
            <font color="#008dc3" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#008dc3" size="5">
                <b>Sociedad Ecuatoriana de Estadística</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#008dc3" size="4">
                Fundamentos de Machine Learning &bull; Validación Cruzada
            </font></p>
            <p style="font-style: oblique;"><font color="#008dc3" size="3">
                Andrés Merino &bull; Febrero 2026
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='008dc3'> Introducción </font>

Este notebook está diseñado como una guía introductoria para implementar la técnica de validación cruzada en la evaluación de modelos de aprendizaje automático.

La idea de la validación cruzada es dividir el conjunto de datos en varias partes (o "folds") y entrenar el modelo en diferentes combinaciones de estas partes para obtener una evaluación más robusta de su rendimiento.

![Validación Cruzada](https://www.analyticslane.com/storage/2018/07/validacion_cruzada.jpeg.webp)

En la práctica, primero dividimos nuestro conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba. Sobre el conjunto de entrenamiento, aplicamos la validación cruzada para evaluar el modelo.

![Validación Cruzada](https://help.qlik.com/es-ES/cloud-services/Subsystems/Hub/Content/Resources/Images/AutomatedMachineLearning/holdout-cross-validation-default.png)


Los paquetes necesarios son:

In [1]:
import pandas as pd  # Manejo de datos

from sklearn.model_selection import train_test_split # División de datos
from sklearn.preprocessing import StandardScaler # Normalización de datos
from sklearn.linear_model import LinearRegression # Modelo de regresión lineal

# Validación cruzada
from sklearn.model_selection import cross_validate, KFold

---
## <font color='008dc3'> Ejemplo práctico </font>


### <font color='008dc3'> Preprocesamiento de datos </font>

Primero leamos los datos y seleccionemos las columnas que utilizaremos:

In [2]:
# Cargar el conjunto de datos
url = 'https://raw.githubusercontent.com/mGalarnyk/Tutorial_Data/master/King_County/kingCountyHouseData.csv'
df = pd.read_csv(url)
# Seleccionar las columnas de interés
columns = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','price'] # Defino las columnas que quiero usar
df = df.loc[:, columns] # Selecciono solo las columnas que quiero usar
display(df.head(10)) # Mostrar las primeras 10 filas del DataFrame

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,price
0,3,1.00,1180,5650,1.0,221900.0
1,3,2.25,2570,7242,2.0,538000.0
2,2,1.00,770,10000,1.0,180000.0
3,4,3.00,1960,5000,1.0,604000.0
4,3,2.00,1680,8080,1.0,510000.0
5,4,4.50,5420,101930,1.0,1225000.0
6,3,2.25,1715,6819,2.0,257500.0
7,3,1.50,1060,9711,1.0,291850.0
8,3,1.00,1780,7470,1.0,229500.0
9,3,2.50,1890,6560,2.0,323000.0


In [3]:
# Dividir el conjunto de datos en características y etiquetas
features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors']
X = df.loc[:, features]
y = df.loc[:, ['price']]

Normalizo los datos.

In [4]:
# Normalizo los datos.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Dividimos los datos en los conjuntos de entrenamiento y prueba.

In [5]:
# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

### <font color='008dc3'> Modelo </font>

Definimos el modelo:

In [6]:
# Crear una regresión lineal
modelo = LinearRegression()
modelo

LinearRegression()

Realizamos validación cruzada:

In [7]:
# Definir los k-folds
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
# Realizar la validación cruzada
resultados = cross_validate(modelo, X_train, y_train, cv=k_fold, scoring='neg_mean_absolute_error', return_train_score=True)

# Mostrar los resultados
resultados = pd.DataFrame(resultados)
display(resultados)

,fit_time,score_time,test_score,train_score
0,0.004005,0.000993,-173364.168098,-171512.078126
1,0.002497,0.000999,-171483.191903,-170437.843327
2,0.002501,0.000501,-176149.671651,-171789.130000
3,0.003002,0.000499,-172274.815747,-171556.225282
4,0.002502,0.000999,-166581.251117,-173750.205216


Mostramos los resultados:

In [8]:
print(f"Precisión promedio en entrenamiento: {resultados['train_score'].mean():.0f} ± {resultados['train_score'].std():.0f}")
print(f"Precisión promedio en prueba: {resultados['test_score'].mean():.0f} ± {resultados['test_score'].std():.0f}")

Precisión promedio en entrenamiento: -171809 ± 1204
Precisión promedio en prueba: -171971 ± 3492


<div style="background-color: #edf1f8; border-color: #008dc3; border-left: 5px solid #008dc3; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
Prueba otros valores de <code>cv</code> y <code>scoring</code> para ver cómo afectan los resultados.
</div>
</br>